# 지도학습을 위한 얕은 방법

아래의 코드로 그래프 기반 지도 머신 러닝을 해결하기 위한 매우 순진하지만 강력한 접근 방식을 탐구한다.
여기서는 사람이 만든 특징 추출의 고전적인 기계 학습 접근 방식에 의존한다.

1장에서 로컬 및 글로벌 그래프 속성을 그래프에서 추출하는 방법을 배웠다.
이러한 속성들은 그래프 분류에 유용할 수 있는 중요한 정보를 제공한다.

In [1]:
!pip install stellargraph

이번 데모에서는 StellarGraph에 있는 PROTEINS 데이터셋을 사용한다.

In [2]:
from stellargraph import datasets
from IPython.display import display, HTML

dataset = datasets.PROTEINS()
display(HTML(dataset.description))
graphs, graph_labels = dataset.load()

각 그래프는 단백질을 나타내고 그래프 라벨은 효소인지 비효소인지를 나타낸다. 이 데이터셋에는 각 그래프에 대해 평균적으로 39개의 노드와 73개의 간선이 있는 1113개의 그래프가 있다. 그래프 노드에는 4개의 속성(라벨의 원-핫 인코딩 포함)이 있으며 각 그래프는 2개의 클래스 중 1개에 속하는 것으로 라벨이 지정된다.

그래프 측정 지표를 계산하기 위한 한 가지 방법은 각 그래프의 인접 행렬 표현을 찾는 것이다.

In [3]:
# StellarGraph 포맷을 numpy 인접 행렬 형태로 변환
adjs = [graph.to_adjacency_matrix().A for graph in graphs]
# 라벨을 Pandas.Series에서 Numpy 행렬로 변환
labels = graph_labels.to_numpy(dtype=int)

In [4]:
import numpy as np
import networkx as nx

metrics = []
for adj in adjs:
    G = nx.from_numpy_matrix(adj)
    # 기본 속성
    num_edges = G.number_of_edges()
    # 클러스터링 측정
    cc = nx.average_clustering(G)
    # 효율성 측정
    eff = nx.global_efficiency(G)

    metrics.append([num_edges, cc, eff])



이제 scikit-learn 유틸리티를 활용하여 훈련과 테스트셋을 생성할 수 있다. 실험에서는 데이터셋의 70%를 훈련 셋으로 사용하고 나머지를 테스트셋으로 사용한다.

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(metrics, labels, test_size=0.3, random_state=42)

많은 기계 학습 워크플로에서 일반적으로 수행되는 것처럼 평균 및 단위 표준 편차가 0이 되도록 특징을 전처리한다.

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

이제 학습을 시작한다. 예제에는 서포트 벡터 머신(SVM)을 선택했다.

In [7]:
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

clf = svm.SVC()
clf.fit(X_train_scaled, y_train)

y_pred = clf.predict(X_test_scaled)

print('Accuracy', accuracy_score(y_test,y_pred))
print('Precision', precision_score(y_test,y_pred))
print('Recall', recall_score(y_test,y_pred))
print('F1-score', f1_score(y_test,y_pred))

Accuracy 0.7455089820359282
Precision 0.7709251101321586
Recall 0.8413461538461539
F1-score 0.8045977011494253


# 그래프 합성곱 신경망을 이용한 지도 그래프 표현 학습

아래의 코드는 딥 그래프 합성곱 신경망을 인코더로 사용하여 지도 그래프 표현 학습을 수행한다.

모델은 그래프 합성곱 레이어를 쌓아서 만들 수 있다.

이번 데모에서는 StellarGraph에 있는 PROTEINS 데이터셋을 사용한다.

In [8]:
import pandas as pd
from stellargraph import datasets
from IPython.display import display, HTML

dataset = datasets.PROTEINS()
display(HTML(dataset.description))
graphs, graph_labels = dataset.load()

labels = graph_labels.to_numpy(dtype=int)

# necessary for converting default string labels to int
graph_labels = pd.get_dummies(graph_labels, drop_first=True)

각 그래프는 단백질을 나타내고 그래프 라벨은 효소인지 비효소인지를 나타낸다. 이 데이터셋에는 각 그래프에 대해 평균적으로 39개의 노드와 73개의 간선이 있는 1113개의 그래프가 있다. 그래프 노드에는 4개의 속성(라벨의 원-핫 인코딩 포함)이 있으며 각 그래프는 2개의 클래스 중 1개에 속하는 것으로 라벨이 지정된다.

모델 구축에 사용하는 StellarGraph는 tf.Keras를 백엔드로 사용한다. 그리고 모델에 학습 데이터를 공급하기 위한 데이터 생성기가 필요하다. 지도 그래프 분류를 위해 StellarGraph의 PaddedGraphGenerator 클래스 인스턴스를 생성한다. 이 생성기는 특징 배열과 인접 행렬을 미니 배치로 Keras 그래프 분류 모델에 제공한다. 노드 수의 차이는 특징 및 인접 행렬의 각 배치를 채우고 유효한 것과 패딩을 나타내는 부울 마스크를 제공한다.

In [9]:
from stellargraph.mapper import PaddedGraphGenerator
generator = PaddedGraphGenerator(graphs=graphs)

이제 실제로 모델을 생성할 준비가 됐다. GCN 레이어는 StellarGraph의 유틸리티 함수를 통해 생성되고 함께 쌓인다. 이 _backbone_ 은 1D Convolutional 레이어와 tf.Keras를 사용하여 완전 연결 레이어(Fully Connected Layer)에 연결된다.

In [10]:
from stellargraph.layer import DeepGraphCNN
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Dropout, Flatten
from tensorflow.keras.losses import binary_crossentropy
import tensorflow as tf

nrows = 35  # the number of rows for the output tensor
layer_dims = [32, 32, 32, 1]

dgcnn_model = DeepGraphCNN(
    layer_sizes=layer_dims,
    activations=["tanh", "tanh", "tanh", "tanh"],
    k=nrows,
    bias=False,
    generator=generator,
)
gnn_inp, gnn_out = dgcnn_model.in_out_tensors()


x_out = Conv1D(filters=16, kernel_size=sum(layer_dims), strides=sum(layer_dims))(gnn_out)
x_out = MaxPool1D(pool_size=2)(x_out)

x_out = Conv1D(filters=32, kernel_size=5, strides=1)(x_out)

x_out = Flatten()(x_out)

x_out = Dense(units=128, activation="relu")(x_out)
x_out = Dropout(rate=0.5)(x_out)

predictions = Dense(units=1, activation="sigmoid")(x_out)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


모델을 컴파일 한다.

In [11]:
model = Model(inputs=gnn_inp, outputs=predictions)
model.compile(optimizer=Adam(lr=0.0001), loss=binary_crossentropy, metrics=["acc"])

70%의 데이터를 학습에 사용하고 나머지는 테스트셋으로 사용한다.

In [12]:
from sklearn import model_selection
train_graphs, test_graphs = model_selection.train_test_split(
    graph_labels, test_size=.3, stratify=labels,
)

In [13]:
gen = PaddedGraphGenerator(graphs=graphs)

train_gen = gen.flow(
    list(train_graphs.index - 1),
    targets=train_graphs.values,
    symmetric_normalization=False,
    batch_size=50,
)

test_gen = gen.flow(
    list(test_graphs.index - 1),
    targets=test_graphs.values,
    symmetric_normalization=False,
    batch_size=1,
)

이제 학습을 시작한다.

In [14]:
epochs = 100
history = model.fit(
    train_gen, epochs=epochs, verbose=1, validation_data=test_gen, shuffle=True,
)

Epoch 1/100


C:\ProgramData\Anaconda3\envs\Graph_ML\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


16/16 [==============================] - 2s 150ms/step - loss: 0.6948 - acc: 0.5379 - val_loss: 0.6533 - val_acc: 0.6467
Epoch 2/100
16/16 [==============================] - 2s 103ms/step - loss: 0.6379 - acc: 0.6508 - val_loss: 0.6185 - val_acc: 0.6467
Epoch 3/100
16/16 [==============================] - 2s 104ms/step - loss: 0.6102 - acc: 0.6611 - val_loss: 0.6061 - val_acc: 0.6796
Epoch 4/100
16/16 [==============================] - 2s 103ms/step - loss: 0.5995 - acc: 0.6970 - val_loss: 0.5966 - val_acc: 0.7246
Epoch 5/100
16/16 [==============================] - 2s 100ms/step - loss: 0.5984 - acc: 0.7150 - val_loss: 0.5889 - val_acc: 0.7305
Epoch 6/100
16/16 [==============================] - 2s 104ms/step - loss: 0.5877 - acc: 0.7330 - val_loss: 0.5856 - val_acc: 0.7335
Epoch 7/100
16/16 [==============================] - 2s 102ms/step - loss: 0.5814 - acc: 0.7202 - val_loss: 0.5814 - val_acc: 0.7275
Epoch 8/100
16/16 [==============================] - 2s 99ms/step - loss: 0.5815 

In [15]:
# https://stellargraph.readthedocs.io/en/stable/demos/graph-classification/index.html

## GraphSAGE를 이용한 지도 노드 표현 학습

In [16]:
from stellargraph import datasets
from IPython.display import display, HTML

dataset = datasets.Cora()
display(HTML(dataset.description))
G, nodes = dataset.load()

Cora 데이터셋은 7개 클래스 중 하나로 분류된 2708개의 과학 출판물로 구성된다. 인용 네트워크는 5429개의 간선으로 구성되어 있다. 데이터 셋의 각 출판물은 사전에서 해당 단어의 부재/존재를 나타내는 0/1 값 단어 벡터로 설명된다. 사전은 1433개의 고유한 단어로 구성되어 있다.

학습(훈련)과 테스트셋으로 나누기

In [17]:
from sklearn.model_selection import train_test_split
train_nodes, test_nodes = train_test_split(
    nodes, train_size=0.1, test_size=None, stratify=nodes
)

범주 분류를 수행하기 때문에 각 범주 라벨을 원 핫 인코딩으로 나타내는 것이 유용하다.

In [18]:
from sklearn import preprocessing, feature_extraction, model_selection
label_encoding = preprocessing.LabelBinarizer()
train_labels = label_encoding.fit_transform(train_nodes)
test_labels = label_encoding.transform(test_nodes)

이제 mdoel을 생성한다. 두 개의 GraphSAGE 레이어와 분류를 위한 softmax 활성화가 있는 Dense 레이어로 구성된다.

In [19]:
from stellargraph.mapper import GraphSAGENodeGenerator
batchsize = 50
n_samples = [10, 5, 7]
generator = GraphSAGENodeGenerator(G, batchsize, n_samples)

In [20]:
from stellargraph.layer import GraphSAGE
from tensorflow.keras.layers import Dense

graphsage_model = GraphSAGE(
    layer_sizes=[32, 32, 16], generator=generator, bias=True, dropout=0.6,
)

In [21]:
gnn_inp, gnn_out = graphsage_model.in_out_tensors()
outputs = Dense(units=train_labels.shape[1], activation="softmax")(gnn_out)

In [22]:
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

model = Model(inputs=gnn_inp, outputs=outputs)
model.compile(optimizer=Adam(lr=0.003), loss=categorical_crossentropy, metrics=["acc"],)

모델에 훈련과 테스트 셋을 공급하기 위해 생성기의 flow 메소드를 사용할 것이다.

In [23]:
train_gen = generator.flow(train_nodes.index, train_labels, shuffle=True)
test_gen = generator.flow(test_nodes.index, test_labels)

이제 학습을 시작한다!

In [24]:
history = model.fit(train_gen, epochs=20, validation_data=test_gen, verbose=2, shuffle=False)

Epoch 1/20
Epoch 1/20
49/6 - 6s - loss: 1.8100 - acc: 0.3060
6/6 - 8s - loss: 1.9074 - acc: 0.1963 - val_loss: 1.8126 - val_acc: 0.3060
Epoch 2/20
Epoch 1/20
49/6 - 6s - loss: 1.7884 - acc: 0.3072
6/6 - 7s - loss: 1.8419 - acc: 0.2852 - val_loss: 1.7864 - val_acc: 0.3072
Epoch 3/20
Epoch 1/20
49/6 - 6s - loss: 1.7452 - acc: 0.3023
6/6 - 7s - loss: 1.7951 - acc: 0.3222 - val_loss: 1.7419 - val_acc: 0.3023
Epoch 4/20
Epoch 1/20
49/6 - 6s - loss: 1.6752 - acc: 0.3027
6/6 - 7s - loss: 1.7446 - acc: 0.3370 - val_loss: 1.6747 - val_acc: 0.3027
Epoch 5/20
Epoch 1/20
49/6 - 6s - loss: 1.6016 - acc: 0.3203
6/6 - 7s - loss: 1.6923 - acc: 0.3519 - val_loss: 1.5992 - val_acc: 0.3203
Epoch 6/20
Epoch 1/20
49/6 - 6s - loss: 1.5288 - acc: 0.4504
6/6 - 7s - loss: 1.6356 - acc: 0.3963 - val_loss: 1.5221 - val_acc: 0.4504
Epoch 7/20
Epoch 1/20
49/6 - 6s - loss: 1.4705 - acc: 0.5369
6/6 - 8s - loss: 1.5830 - acc: 0.4704 - val_loss: 1.4651 - val_acc: 0.5369
Epoch 8/20
Epoch 1/20
49/6 - 6s - loss: 1.4463 -